TITLE: Analysis of User Knowledge Modeling: Predicting an individuals intelligence level

INTRODUCTION: Understanding knowledge is key for assessing an individual's ability to learn and apply new information. Generally, this is a challenging problem due to the fact that many people are skilled in varied fields, and thus don't necessarily have a clear idea as to how they stack up among others. Using different classifications ranging from "very_low" to "high", This data set contains a variety of potential predictor values that may help us predict an individual's knowledge level. In this analysis, we'll ask the following question: "Can STG (Study Time for Goal) and PEG (Exam Performance for Goal) act as predictor values to help us figure out a user's knowledge level?" We'll use the following data set: https://archive.ics.uci.edu/static/public/257/user+knowledge+modeling.zip